In [8]:
import cv2
from tqdm import tqdm
!pip install ultralytics
from ultralytics import YOLO
import torchvision
import logging
import gc

In [9]:
## HERE IS THE INPUT VIDEO. SET THIS
input_video_path = 'Backyard-picnic.mp4'

## OUTPUT FILES. AUTOMATICALLY SET.
output_video_path = input_video_path+"_out.avi"
output_text_file=input_video_path+"_out.txt"

In [10]:
# Set up logging
logging.basicConfig(
    level=logging.DEBUG,
    filename='video_labeler.log',
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s')
console = logging.StreamHandler()
console.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

In [11]:
try:
    # Load YOLOv8 model
    model = YOLO("yolov8n.pt")  # You can replace "YOLOv8x.pt"  with other YOLOv8 models like yolov8s.pt, etc. https://docs.ultralytics.com/tasks/detect/#models
    logging.info("YOLOv8 model loaded successfully.")

    # Load video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise Exception(
            f"Error: Could not open input video file: {input_video_path}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    out = cv2.VideoWriter(
        output_video_path, fourcc, fps, (width, height))
    if not out.isOpened():
        raise Exception(
            f"Error: Could not open output video file for writing: {output_video_path}")

    logging.info(
        f"VideoWriter opened successfully: {out.isOpened()}")

    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Open the text file to write detected objects
    with open(output_text_file, "w") as file, tqdm(total=total_frames,
         desc="Processing frames") as pbar:
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Detecting objects using YOLOv8
            try:
                results = model(frame, device='cpu')  # Use CPU for inference
                detected_objects = []

                for result in results:
                    boxes = result.boxes  # Boxes object for bbox outputs

                    if len(boxes.xyxy) > 0:
                        boxes_tensor = boxes.xyxy.to('cpu')
                        scores = boxes.conf.to('cpu')
                        nms_indices = torchvision.ops.nms(boxes_tensor, scores, 0.5)

                        for idx in nms_indices:
                            x1, y1, x2, y2 = boxes.xyxy[idx].tolist()
                            confidence = boxes.conf[idx].item()
                            class_id = int(boxes.cls[idx].item())
                            label = model.names[class_id]
                            detected_objects.append(label)

                            # Draw bounding box and label on the frame
                            color = (0, 255, 0)
                            cv2.rectangle(frame,
                                          (int(x1), int(y1)),
                                          (int(x2), int(y2)),
                                          color, 2)
                            cv2.putText(frame, label,
                                        (int(x1), int(y1) - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                out.write(frame)  # Write the frame to the video file

                # Write detected objects to file
                file.write(
                    f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: {', '.join(detected_objects)}\n")
                file.flush()  # Flush the file buffer to ensure data is written to disk


                frame_count += 1
                pbar.update(1)  # Update the progress bar

                # Debugging output every 100 frames
                if frame_count % 100 == 0:
                    logging.info(f"Processed {frame_count}/{total_frames} frames.")

                # Clear memory
                del results, detected_objects
                gc.collect()

            except Exception as e:
                logging.error(f"Error processing frame {frame_count}: {e}")
                continue

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print("Output video file: ", output_video_path)
    print("Output object list file: ", output_text_file)

    logging.info("Processing complete. Output video and detected objects file have been created.")
except Exception as e:
    logging.error(f"An error occurred: {e}")


Processing frames:   0%|          | 0/924 [00:00<?, ?it/s]


0: 384x640 (no detections), 319.2ms
Speed: 17.7ms preprocess, 319.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:   0%|          | 1/924 [00:01<26:37,  1.73s/it]


0: 384x640 (no detections), 228.7ms
Speed: 5.8ms preprocess, 228.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:   0%|          | 2/924 [00:02<16:18,  1.06s/it]

...


0: 384x640 1 person, 385.0ms
Speed: 23.6ms preprocess, 385.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)


Processing frames: 100%|█████████▉| 923/924 [06:50<00:00,  1.24it/s]


0: 384x640 1 person, 281.7ms
Speed: 8.0ms preprocess, 281.7ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)


Processing frames: 100%|██████████| 924/924 [06:51<00:00,  2.25it/s]

Output video file:  Backyard-picnic.mp4_out.avi
Output object list file:  Backyard-picnic.mp4_out.txt
